In [11]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

def Sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return(y)

def ReLU(x):
    return max(0, x)

def softmax(x):
    ex = np.exp(x)
    y = ex / np.sum(ex)
    return(y)

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})

# ReLU Function

In [12]:
def ReLU(x):
    return np.maximum(0, x)

def Softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def DeepReLU(W1, W2, W3, W4, X, D):
    alpha = 0.01
    N = 5
    for k in range(N):
        x = X[:, :, k].reshape(25, 1)
        v1 = np.dot(W1, x)
        y1 = ReLU(v1)
        v2 = np.dot(W2, y1)
        y2 = ReLU(v2)
        v3 = np.dot(W3, y2)
        y3 = ReLU(v3)
        v = np.dot(W4, y3)
        y = Softmax(v)
        d = D[k, :].reshape(-1, 1)
        e = d - y
        delta = e
        e3 = np.dot(W4.T, delta)
        delta3 = (v3 > 0) * e3
        e2 = np.dot(W3.T, delta3)
        delta2 = (v2 > 0) * e2
        e1 = np.dot(W2.T, delta2)
        delta1 = (v1 > 0) * e1
        dW4 = alpha * np.dot(delta, y3.T)
        W4 = W4 + dW4
        dW3 = alpha * np.dot(delta3, y2.T)
        W3 = W3 + dW3
        dW2 = alpha * np.dot(delta2, y1.T)
        W2 = W2 + dW2
        dW1 = alpha * np.dot(delta1, x.T)
        W1 = W1 + dW1
    return W1, W2, W3, W4

X = np.zeros((5, 5, 5))
X[:, :, 0] = np.array([[0, 1, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 1, 1, 1, 0]])
X[:, :, 1] = np.array([[1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [0, 1, 1, 1, 0],
                       [1, 0, 0, 0, 0],
                       [1, 1, 1, 1, 1]])
X[:, :, 2] = np.array([[1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [0, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [1, 1, 1, 1, 0]])
X[:, :, 3] = np.array([[0, 0, 0, 1, 0],
                       [0, 0, 1, 1, 0],
                       [0, 1, 0, 1, 0],
                       [1, 1, 1, 1, 1],
                       [0, 0, 0, 1, 0]])
X[:, :, 4] = np.array([[1, 1, 1, 1, 1],
                       [1, 0, 0, 0, 0],
                       [1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [1, 1, 1, 1, 0]])
D = np.eye(5)
W1 = 2 * np.random.rand(20, 25) - 1
W2 = 2 * np.random.rand(20, 20) - 1
W3 = 2 * np.random.rand(20, 20) - 1
W4 = 2 * np.random.rand(5, 20) - 1

for epoch in range(10000):  # train
    W1, W2, W3, W4 = DeepReLU(W1, W2, W3, W4, X, D)

N = 5  # inference
predict = np.array([])
for k in range(N):
    x = X[:, :, k].reshape(25, 1)
    v1 = np.dot(W1, x)
    y1 = ReLU(v1)
    v2 = np.dot(W2, y1)
    y2 = ReLU(v2)
    v3 = np.dot(W3, y2)
    y3 = ReLU(v3)
    v = np.dot(W4, y3)
    y = Softmax(v)
    predict = np.append(predict, y)
    
predict

array([1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000, 0.000, 0.000,
       0.000, 0.000, 0.000, 1.000, 0.000, 0.000, 0.000, 0.000, 0.000,
       1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000])

In [13]:
predict.reshape(5,5)

array([[1.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 1.000, 0.000, 0.000, 0.000],
       [0.000, 0.000, 1.000, 0.000, 0.000],
       [0.000, 0.000, 0.000, 1.000, 0.000],
       [0.000, 0.000, 0.000, 0.000, 1.000]])

# Dropout

In [14]:
import numpy as np

def Dropout(x, dropout_rate):
    mask = np.random.rand(*x.shape) > dropout_rate
    return x * mask


def DeepDropout(W1, W2, W3, W4, X, D):
    alpha = 0.01
    N = 5
    for k in range(N):
        x = X[:, :, k].reshape(25, 1)
        v1 = np.dot(W1, x)
        y1 = Sigmoid(v1)
        y1 = y1 * Dropout(y1, 0.2)
        v2 = np.dot(W2, y1)
        y2 = Sigmoid(v2)
        y2 = y2 * Dropout(y2, 0.2)
        v3 = np.dot(W3, y2)
        y3 = Sigmoid(v3)
        y3 = y3 * Dropout(y3, 0.2)
        v = np.dot(W4, y3)
        y = Softmax(v)
        d = D[k, :].reshape(-1, 1)
        e = d - y
        delta = e
        e3 = np.dot(W4.T, delta)
        delta3 = y3 * (1 - y3) * e3
        e2 = np.dot(W3.T, delta3)
        delta2 = y2 * (1 - y2) * e2
        e1 = np.dot(W2.T, delta2)
        delta1 = y1 * (1 - y1) * e1
        dW4 = alpha * np.dot(delta, y3.T)
        W4 = W4 + dW4
        dW3 = alpha * np.dot(delta3, y2.T)
        W3 = W3 + dW3
        dW2 = alpha * np.dot(delta2, y1.T)
        W2 = W2 + dW2
        dW1 = alpha * np.dot(delta1, x.T)
        W1 = W1 + dW1
    return W1, W2, W3, W4

X = np.zeros((5, 5, 5))
X[:, :, 0] = np.array([[0, 1, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 0, 1, 0, 0],
                       [0, 1, 1, 1, 0]])
X[:, :, 1] = np.array([[1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [0, 1, 1, 1, 0],
                       [1, 0, 0, 0, 0],
                       [1, 1, 1, 1, 1]])
X[:, :, 2] = np.array([[1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [0, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [1, 1, 1, 1, 0]])
X[:, :, 3] = np.array([[0, 0, 0, 1, 0],
                       [0, 0, 1, 1, 0],
                       [0, 1, 0, 1, 0],
                       [1, 1, 1, 1, 1],
                       [0, 0, 0, 1, 0]])
X[:, :, 4] = np.array([[1, 1, 1, 1, 1],
                       [1, 0, 0, 0, 0],
                       [1, 1, 1, 1, 0],
                       [0, 0, 0, 0, 1],
                       [1, 1, 1, 1, 0]])
D = np.eye(5)
W1 = 2 * np.random.rand(20, 25) - 1
W2 = 2 * np.random.rand(20, 20) - 1
W3 = 2 * np.random.rand(20, 20) - 1
W4 = 2 * np.random.rand(5, 20) - 1

for epoch in range(10000):  # train
    W1, W2, W3, W4 = DeepDropout(W1, W2, W3, W4, X, D)

n = 5  # inference
predict = np.array([])
for k in range(n):
    x = X[:, :, k].reshape(25, 1)
    v1 = np.dot(W1, x)
    y1 = Sigmoid(v1)
    v2 = np.dot(W2, y1)
    y2 = Sigmoid(v2)
    v3 = np.dot(W3, y2)
    y3 = Sigmoid(v3)
    v = np.dot(W4, y3)
    y = Softmax(v)
    predict = np.append(predict, y)
    
predict.reshape(5,5)

array([[1.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 0.999, 0.000, 0.000, 0.000],
       [0.000, 0.000, 1.000, 0.000, 0.000],
       [0.000, 0.000, 0.000, 1.000, 0.000],
       [0.000, 0.000, 0.001, 0.000, 0.999]])